# Implementando la BBDD de Datos de Mercado en PostgreSQL

En la sesión creamos una definición más o menos precisa para una BBDD de precios de mercado. En esta sesión vamos a implementar esa BBDD en un servidor PostreSQL en la nube.

## PostreSQL

PostgreSQL es un motor de bases de datos relacional open source muy utilizado tanto en otros proyectos open source como en entornos corporativos. Es un motor muy avanzado y de alto rendimiento. Para más detalles ver https://www.postgresql.org/.

Para esta actividad contamos con un servidor PostgreSQL en la nube. Vamos a establecer conexión con él, para eso, necesitamos la librería en Python que permite la interacción con un motor de BBDD PostreSQL.

In [1]:
import psycopg2

Se establece la conexión.

In [2]:
conn = psycopg2.connect(
    host="chunee.db.elephantsql.com",
    database="fzvvearc",
    user="fzvvearc",
    password="U2kRI8TA45Ie6zm3GIsM8D0-aZWZXTDN"
)

### Revisando la BBDD

Esta BBDD tiene ya algunas cosas que yo he creado para probar la conexión y la performance. Investiguemos un poco:

In [3]:
qry_schemas = "select schema_name from information_schema.schemata"
with conn.cursor() as cursor:
    cursor.execute(qry_schemas)
    data = cursor.fetchall()

In [4]:
data

[('pg_catalog',),
 ('public',),
 ('information_schema',),
 ('fixed_income',),
 ('market_data',),
 ('calendar_data',),
 ('enum_types',),
 ('user_data',)]

También podemos usar `pandas` para tener un ouput más amistoso.

In [5]:
import pandas as pd

In [6]:
pd.read_sql(qry_schemas, conn)

,schema_name
0,pg_catalog
1,public
2,information_schema
3,fixed_income
4,market_data
5,calendar_data
6,enum_types
7,user_data


Los últimos 5 esquemas, son los que yo creé. Vamos a investigar un poco más.

**NOTA:** Los esquemas (*schema*) de una BBDD son simplemente grupos de tablas. Es una agrupación que se realiza por diseño y no es estrictamente necesaria. Es similar a la organización en carpetas de los archivos en un disco duro.

In [7]:
qry = "SELECT table_schema, table_name FROM information_schema.tables WHERE table_schema = 'market_data'"
pd.read_sql(qry, conn)

,table_schema,table_name
0,market_data,capitalization_indexes
1,market_data,capitalization_index_values
2,market_data,currencies
3,market_data,fx_rates
4,market_data,fx_rate_indexes
5,market_data,fx_rate_index_values


Vamos a ver que hay en la tabla `fx_rate_index_values`.

In [8]:
qry = "SELECT * FROM market_data.fx_rate_index_values"
result = pd.read_sql(qry, conn)

In [9]:
result.head()

,id,fx_rate_index_code,date,value
0,1,UF,2000-01-01,15067.93
1,2,UF,2000-01-02,15068.90
2,3,UF,2000-01-03,15069.87
3,4,UF,2000-01-04,15070.84
4,5,UF,2000-01-05,15071.82


In [10]:
result.tail()

,id,fx_rate_index_code,date,value
7947,7948,UF,2021-10-05,30108.40
7948,7949,UF,2021-10-06,30112.40
7949,7950,UF,2021-10-07,30116.41
7950,7951,UF,2021-10-08,30120.42
7951,7952,UF,2021-10-09,30124.43


El valor de la UF desde el 2000 hasta hoy ... en 21 años se duplicó el valor de la UF.

$$UF_T=UF_0\cdot\left(1+I_{anual}\right)^{21}$$

$$I_{anual}=\left(\frac{UF_T}{UF_0}\right)^{\frac{1}{21}}-1$$

Haciendo el cálculo muy al ojo ...

In [11]:
i = (30123.33 / 15067.93)**(1 / 21) - 1
print(f'Inflación anual promedio de los últimos 21 años: {i:.2%}')

Inflación anual promedio de los últimos 21 años: 3.35%


## Construyendo la Nueva BBDD

Partamos creando un esquema donde iremos a alojar las tablas que definimos.

In [12]:
qry = "CREATE SCHEMA IF NOT EXISTS lv_cdb_mkt_data"
with conn.cursor() as cursor:
    cursor.execute(qry)
    conn.commit()

Verifiquemos.

In [13]:
pd.read_sql(qry_schemas, conn)

,schema_name
0,pg_catalog
1,public
2,information_schema
3,fixed_income
4,market_data
5,calendar_data
6,enum_types
7,user_data
8,lv_cdb_mkt_data


### Tablas

Las siguientes son las tablas definidas en la sesión anterior, pero ordenadas de forma que podamos ir dándolas de alta una a una.

- Tipo de valor
  - id
  - codigo (bid, ask, ...)
  - descripcion 
 
 
 - Tipo de dato
  - id
  - codigo (unique)
  - descripcion
  
  
- Asset Class
 - id
 - codigo
 - descripcion


 - Activos
  - id
  - codigo
  - codigo asset class
  - descripcion
 
 
- Caracteristicas Activos
 - id
 - codigo del activo (FK a Activos)
 - characteristicas: y esto es un JSON (piensen en un Dict de Python)

  
- Serie de precios (precios diarios):
 - timestamp
 - identificador del activo (FK a Activos)
 - tipo de dato (FK al codigo de tipo de dato)
 - tipo de valor (bid, ask, close, last ..., NAV que aplica para los ETF)
 - valor
 

- Curvas - Superficies
 - id
 - codigo
 - descripcion


- Elementos Curvas
 - id
 - codigo activo
 - codigo curva
 

- Emisores


- Ratings
 - Necesita una dimensión temporal

### Tabla `tipo de valor`

Los "\" son sólo para continuar la `str` en la línea siguiente, no son parte de la query ejecutada.

In [14]:
qry = "CREATE TABLE IF NOT EXISTS lv_cdb_mkt_data.tipo_de_valor ( \
id serial8 PRIMARY KEY, \
codigo VARCHAR ( 50 ) UNIQUE NOT NULL, \
descripcion VARCHAR ( 255 ) NOT NULL);"
with conn.cursor() as cursor:
    cursor.execute(qry)
    conn.commit()

In [15]:
print(qry)

CREATE TABLE IF NOT EXISTS lv_cdb_mkt_data.tipo_de_valor ( id serial8 PRIMARY KEY, codigo VARCHAR ( 50 ) UNIQUE NOT NULL, descripcion VARCHAR ( 255 ) NOT NULL);


### Tabla `tipo_de_dato`

In [16]:
qry = "CREATE TABLE IF NOT EXISTS lv_cdb_mkt_data.tipo_de_dato ( \
id serial8 PRIMARY KEY, \
codigo VARCHAR ( 50 ) UNIQUE NOT NULL, \
descripcion VARCHAR ( 255 ) NOT NULL);"
with conn.cursor() as cursor:
    cursor.execute(qry)
    conn.commit()

### Tabla `clase_de_activo` (asset class)

In [17]:
qry = "CREATE TABLE IF NOT EXISTS lv_cdb_mkt_data.clase_de_activo ( \
id serial8 PRIMARY KEY, \
codigo VARCHAR ( 50 ) UNIQUE NOT NULL, \
descripcion VARCHAR ( 255 ) NOT NULL);"
with conn.cursor() as cursor:
    cursor.execute(qry)
    conn.commit()

### Tabla `activos`

In [18]:
qry = "CREATE TABLE IF NOT EXISTS lv_cdb_mkt_data.activos ( \
id serial8 PRIMARY KEY, \
codigo VARCHAR ( 50 ) UNIQUE NOT NULL, \
descripcion VARCHAR ( 255 ) NOT NULL, \
codigo_clase_activo VARCHAR ( 50 ) NOT NULL, \
caracteristicas VARCHAR ( 255 ) NOT NULL, \
FOREIGN KEY (codigo_clase_activo) \
REFERENCES lv_cdb_mkt_data.clase_de_activo (codigo));"
with conn.cursor() as cursor:
    cursor.execute(qry)
    conn.commit()

### Tabla `caracteristicas_activos`

In [19]:
qry = "CREATE TABLE IF NOT EXISTS lv_cdb_mkt_data.caracteristicas_activos ( \
id serial8 PRIMARY KEY, \
codigo_activo VARCHAR ( 50 ) NOT NULL, \
caracteristicas JSONB NOT NULL, \
FOREIGN KEY (codigo_activo) \
REFERENCES lv_cdb_mkt_data.activos (codigo));"
with conn.cursor() as cursor:
    cursor.execute(qry)
    conn.commit()

### Tabla `precios` (series de precios)

In [20]:
qry = "CREATE TABLE IF NOT EXISTS lv_cdb_mkt_data.precios ( \
id serial8 PRIMARY KEY, \
fecha_hora TIMESTAMP NOT NULL, \
codigo_activo VARCHAR ( 50 ) NOT NULL, \
codigo_tipo_de_dato VARCHAR ( 50 ) NOT NULL, \
codigo_tipo_de_valor VARCHAR ( 50 ) NOT NULL, \
valor NUMERIC NOT NULL, \
FOREIGN KEY (codigo_activo) \
REFERENCES lv_cdb_mkt_data.activos (codigo), \
FOREIGN KEY (codigo_tipo_de_dato) \
REFERENCES lv_cdb_mkt_data.tipo_de_dato (codigo), \
FOREIGN KEY (codigo_tipo_de_valor) \
REFERENCES lv_cdb_mkt_data.tipo_de_valor (codigo));"
with conn.cursor() as cursor:
    cursor.execute(qry)
    conn.commit()

### Tarea

Dar de alta las tablas que faltan.

## Insertar Data

Vamos a subir algunos valores de la UF para partir. La data es la siguiente:

In [21]:
ufs = pd.read_csv('data/valores_uf.csv')

In [22]:
ufs.head()

,fx_rate_index_code,date,value
0,UF,2021-10-09,30124.43
1,UF,2021-10-08,30120.42
2,UF,2021-10-07,30116.41
3,UF,2021-10-06,30112.40
4,UF,2021-10-05,30108.40


Pero no es llegar y subirla, hay que seguir el orden de las tablas que dimos de alta.

- Clase de activo ---> INDICE
- El activo ---> UF
- Tipo de valor ---> DIARIO
- Tipo de dato ---> PUBLICADO

Vamos a definir funciones para hacer las inserciones, así no tenemos que recordar la sintaxis SQL.

### `clase_activo`

In [23]:
def inserta_clase_activo(codigo, descripcion, conn):
    qry = "INSERT INTO lv_cdb_mkt_data.clase_de_activo (codigo, descripcion) \
    VALUES (%s, %s)"
    try:
        cursor = conn.cursor()
        cursor.execute(qry, [codigo, descripcion])
        conn.commit()
    except Exception as e:
        print(f'Se produjo el error: {str(e)}')
    finally:
        conn.rollback()
        if cursor is not None:
            cursor.close()

In [24]:
inserta_clase_activo('INDICE', 'Representa índices de inflación, equity, tasa de interés ...', conn)

### `activo`

In [25]:
def inserta_activo(codigo, descripcion, clase_activo, caracteristicas, conn):
    qry = "INSERT INTO lv_cdb_mkt_data.activos (codigo, descripcion, \
    codigo_clase_activo, caracteristicas) \
    VALUES (%s, %s, %s, %s)"
    try:
        cursor = conn.cursor()
        cursor.execute(qry, [codigo, descripcion, clase_activo, caracteristicas])
        conn.commit()
    except Exception as e:
        print(f'Se produjo el error: {str(e)}')
    finally:
        conn.rollback()
        if cursor is not None:
            cursor.close()

In [26]:
inserta_activo('UF', 'Indice de inflación mensual de Chile', 'INDICE', '{}', conn)

### `tipo_de_valor`

In [27]:
def inserta_tipo_valor(codigo, descripcion, conn):
    qry = "INSERT INTO lv_cdb_mkt_data.tipo_de_valor (codigo, descripcion) \
    VALUES (%s, %s)"
    try:
        cursor = conn.cursor()
        cursor.execute(qry, [codigo, descripcion])
        conn.commit()
    except Exception as e:
        print(f'Se produjo el error: {str(e)}')
    finally:
        conn.rollback()
        if cursor is not None:
            cursor.close()

In [28]:
inserta_tipo_valor('DIARIO', 'Valor que se registra una vez al día', conn)

### `tipo_de_dato`

In [29]:
def inserta_tipo_dato(codigo, descripcion, conn):
    qry = "INSERT INTO lv_cdb_mkt_data.tipo_de_dato (codigo, descripcion) \
    VALUES (%s, %s)"
    try:
        cursor = conn.cursor()
        cursor.execute(qry, [codigo, descripcion])
        conn.commit()
    except Exception as e:
        print(f'Se produjo el error: {str(e)}')
    finally:
        conn.rollback()
        if cursor is not None:
            cursor.close()

In [30]:
inserta_tipo_dato(
    'PUBLICADO',
    'Representa el valor oficial publicado por el ente responsable del valor',
    conn
)

### `precios`

In [36]:
def inserta_precios(data, conn):
    qry = "INSERT INTO lv_cdb_mkt_data.precios (fecha_hora, codigo_activo, \
    codigo_tipo_de_dato, codigo_tipo_de_valor, valor) VALUES \
    (%s, %s, %s, %s, %s)"
    try:
        cursor = conn.cursor()
        cursor.executemany(qry, data) # executemany requiere una matriz de datos
        conn.commit()
    except Exception as e:
        print(f'Se produjo el error: {str(e)}')
    finally:
        conn.rollback()
        if cursor is not None:
            cursor.close()

In [37]:
ufs['tipo_valor'] = 'DIARIO'
ufs['tipo_dato'] = 'PUBLICADO'
ufs

,fx_rate_index_code,date,value,tipo_valor,tipo_dato
0,UF,2021-10-09,30124.43,DIARIO,PUBLICADO
1,UF,2021-10-08,30120.42,DIARIO,PUBLICADO
2,UF,2021-10-07,30116.41,DIARIO,PUBLICADO
3,UF,2021-10-06,30112.40,DIARIO,PUBLICADO
4,UF,2021-10-05,30108.40,DIARIO,PUBLICADO
...,...,...,...,...,...
495,UF,2020-06-01,28716.52,DIARIO,PUBLICADO
496,UF,2020-05-31,28716.52,DIARIO,PUBLICADO
497,UF,2020-05-30,28716.52,DIARIO,PUBLICADO
498,UF,2020-05-29,28716.52,DIARIO,PUBLICADO


Reordenamos las columnas.

In [38]:
ufs = ufs[['date', 'fx_rate_index_code', 'tipo_dato', 'tipo_valor', 'value']]
ufs

,date,fx_rate_index_code,tipo_dato,tipo_valor,value
0,2021-10-09,UF,PUBLICADO,DIARIO,30124.43
1,2021-10-08,UF,PUBLICADO,DIARIO,30120.42
2,2021-10-07,UF,PUBLICADO,DIARIO,30116.41
3,2021-10-06,UF,PUBLICADO,DIARIO,30112.40
4,2021-10-05,UF,PUBLICADO,DIARIO,30108.40
...,...,...,...,...,...
495,2020-06-01,UF,PUBLICADO,DIARIO,28716.52
496,2020-05-31,UF,PUBLICADO,DIARIO,28716.52
497,2020-05-30,UF,PUBLICADO,DIARIO,28716.52
498,2020-05-29,UF,PUBLICADO,DIARIO,28716.52


In [39]:
inserta_precios(ufs.values, conn)